In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
import datetime
from dotenv import load_dotenv

model = OllamaLLM(model="gemma3:1b")


In [2]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system","你是一個專業的客服代表，負責回覆客戶郵件。請用友善、專業的語調回覆。請不要回覆(公司的資訊和結尾詞)，我會自己手動增加請不要回覆公司的資訊，我會自己手動增加"),
    ("human", "請回覆這封客戶郵件：\n\n{email_content}")
])

In [3]:
def format_email_reply(ai_answer:str) -> str:
    formatted_reply = f"""{ai_answer}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
客服團隊

回復時間:{datetime.datetime.now().strftime("%Y-%m-%d %H:%M")}"""

    return {"formatted_reply":formatted_reply}
    
format_email = RunnableLambda(format_email_reply)

In [4]:
chain = (prompt_template | model | StrOutputParser() | format_email)

In [5]:
customer_email = """您好:

我最近購買了貴公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎？我已經購買一週了。

期待您的回覆。

王小明
"""

In [6]:
result = chain.invoke({"email_content": customer_email})
print(result)

{'formatted_reply': '您好，王小明您好！非常抱歉讓您遇到這樣的狀況。我們深感遺憾，聽到您產品包裝損壞且功能與說明不符，確實讓您感到困擾。\n\n針對您的問題，我們非常樂意為您處理。由於您已經購買一週了，您可以透過以下方式進行退貨或換貨：\n\n*   您可以聯繫我們的客服專線 [客服電話號碼]，我們將協助您處理退貨或換貨。\n*   如果您更喜歡，也可以直接向我們提出您的退貨申請，我們將盡力協助您。\n\n請您放心，我們會盡力協助您解決問題，讓您獲得滿意的解決方案。\n\n期待您的回覆，我們希望能盡可能地為您解決這個問題。\n\n感謝您的來信，如有其他問題請隨時聯繫我們。\n\n此致\n客服團隊\n\n回復時間:2025-10-18 15:16'}
